In [2]:
!pwd

/Users/Pavlo/PycharmProjects/ADL-Minicar-Challenge-2023/mycar/notebooks


In [1]:
import os
import time
import tensorflow as tf
import numpy as np

print(tf.__version__)

2.2.0


In [2]:
!which python

/gpfs/space/software/jupyterhub/python/jupyter/bin/python


In [5]:
ls ../models

pedestrian_detection_v3/  stop_sign_detector/


In [39]:
model_path = "../models/stop_sign_detector_v3"
tflite_model_path = "../models/stop_sign_detector_v3.tflite"

In [34]:
# Workaround for this bug https://github.com/tensorflow/tensorflow/issues/34698
bin_path = os.path.dirname(os.sys.executable)
if 'PATH' in os.environ:
    os.environ['PATH'] += ':' + bin_path
else:
    os.environ['PATH'] = bin_path
print(bin_path)
print(os.environ['PATH'])

/gpfs/space/home/pyvovar/miniconda3/envs/donkey/bin
/gpfs/space/software/cluster_software/spack/linux-centos7-x86_64/gcc-9.2.0/git-2.35.2-pkznwh5mpp74zq6p42b47pxpgbxxrgwk/bin:/gpfs/space/software/cluster_software/spack/linux-centos7-x86_64/gcc-9.2.0/perl-5.34.1-4ryb2w6tlh6ha54x25kkpq7l5zjq56do/bin:/gpfs/space/software/cluster_software/spack/linux-centos7-x86_64/gcc-9.2.0/pcre2-10.36-viek2bpe7b5ktcr3qpyl2qhipy2ybjxs/bin:/gpfs/space/software/cluster_software/spack/linux-centos7-x86_64/gcc-9.2.0/openssh-9.0p1-xuo4l7k2kii6nqw2x2t22jqghijrsdgi/bin:/gpfs/space/software/cluster_software/spack/linux-centos7-x86_64/gcc-9.2.0/curl-7.83.0-yzcvzf3nqftyqhrki2zketnom3lbg3kf/bin:/gpfs/space/software/cluster_software/spack/linux-centos7-x86_64/gcc-9.2.0/openssl-1.1.1o-m6uvz6ittrnhnwd3oftfljxqwkve44j7/bin:/gpfs/space/software/cluster_software/spack/linux-centos7-x86_64/gcc-9.2.0/libidn2-2.3.0-qmmoo5lcgqh4crq2cbbwqe26b6ujugzx/bin:/gpfs/space/software/cluster_software/spack/linux-centos7-x86_64/gcc-9.2.0

In [40]:
converter = tf.lite.TFLiteConverter.from_saved_model(model_path)
tflite_model = converter.convert()
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

2022-12-07 14:23:32.431494: I tensorflow/core/grappler/devices.cc:60] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA support)
2022-12-07 14:23:32.431625: I tensorflow/core/grappler/clusters/single_machine.cc:356] Starting new session
2022-12-07 14:23:32.435733: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:797] Optimization results for grappler item: graph_to_optimize
2022-12-07 14:23:32.435781: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:799]   function_optimizer: Graph size after: 57 nodes (44), 79 edges (66), time = 1.957ms.
2022-12-07 14:23:32.435789: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:799]   function_optimizer: function_optimizer did nothing. time = 0.058ms.
2022-12-07 14:23:32.522116: I tensorflow/core/grappler/devices.cc:60] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA support)
2022-12-07 14:23:32

In [41]:
model = tf.keras.models.load_model(model_path)

input_data = np.array(np.random.random_sample([1, 120, 160, 3]), dtype=np.float32)
print(input_data.shape)

start = time.perf_counter()

output_data = model(input_data)

stop = time.perf_counter()

print(output_data)
print(f"Elapsed Time: {stop - start} ms")

(1, 120, 160, 3)
tf.Tensor([[0.99968994]], shape=(1, 1), dtype=float32)
Elapsed Time: 0.009328829124569893 ms


In [42]:
# Load the TFLite model and allocate tensors
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the model on random input data
input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
print(input_data.shape)

# Inference
start = time.perf_counter()

interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

# get_tensor() returns a copy of the tensor data
# use tensor() in order to get a pointer to the tensor
output_data = interpreter.get_tensor(output_details[0]['index'])

stop = time.perf_counter()

print(output_data)
print(f"Elapsed Time: {stop - start} ms")

(1, 120, 160, 3)
[[0.9997409]]
Elapsed Time: 0.025668388232588768 ms
